In [1]:

import os
import cv2
import albumentations as A
import numpy as np

# Paths
image_dir = "/mnt/nvme_disk2/User_data/hp927k/phase2/BBPrediction/images/train"
label_dir = "/mnt/nvme_disk2/User_data/hp927k/phase2/BBPrediction/labels/train"
output_image_dir = "/mnt/nvme_disk2/User_data/hp927k/phase2/BBPrediction/images/train_augmented"
output_label_dir = "/mnt/nvme_disk2/User_data/hp927k/phase2/BBPrediction/labels/train_augmented"

# Create output directories if they don't exist
os.makedirs(output_image_dir, exist_ok=True)
os.makedirs(output_label_dir, exist_ok=True)

# Define various augmentation pipelines
augmentations = {
    "brightness_contrast": A.Compose([A.RandomBrightnessContrast(p=1.0)], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels'])),
    # "rotation": A.Compose([A.Rotate(limit=30, p=1.0)], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels'])),
    "blur": A.Compose([A.Blur(blur_limit=5, p=1.0)], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels'])),
    "clahe": A.Compose([A.CLAHE(p=1.0)], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels'])),
    "scaling": A.Compose([A.RandomScale(scale_limit=0.2, p=1.0)], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels'])),
}

def read_yolo_label(label_path):
    """Read YOLO label file and return bounding boxes and class labels."""
    with open(label_path, 'r') as f:
        lines = f.read().strip().split('\n')
        bboxes = []
        class_labels = []
        for line in lines:
            parts = line.split()
            class_id = int(parts[0])
            bbox = [float(x) for x in parts[1:]]
            bboxes.append(bbox)
            class_labels.append(class_id)
        return bboxes, class_labels

def write_yolo_label(label_path, bboxes, class_labels):
    """Write bounding boxes and class labels to a YOLO label file."""
    with open(label_path, 'w') as f:
        for bbox, class_id in zip(bboxes, class_labels):
            f.write(f"{class_id} {' '.join(map(str, bbox))}\n")

# Loop through all images and labels
for image_name in os.listdir(image_dir):
    if image_name.endswith('.jpg') or image_name.endswith('.png'):
        image_path = os.path.join(image_dir, image_name)
        label_path = os.path.join(label_dir, image_name.replace('.jpg', '.txt').replace('.png', '.txt'))
        print(f"Processing: {image_name}")

        # Check if both image and label exist
        if not os.path.exists(image_path) or not os.path.exists(label_path):
            print(f"Skipping {image_name}: Image or label file missing.")
            continue

        # Read image and label
        image = cv2.imread(image_path)
        if image is None:
            print(f"Skipping {image_name}: Unable to read image.")
            continue

        bboxes, class_labels = read_yolo_label(label_path)
       

        # Save the original image and label to the output directory
        output_image_path = os.path.join(output_image_dir, image_name)
        output_label_path = os.path.join(output_label_dir, image_name.replace('.jpg', '.txt').replace('.png', '.txt'))
        cv2.imwrite(output_image_path, image)
        write_yolo_label(output_label_path, bboxes, class_labels)

        # Apply each augmentation
        for aug_name, transform in augmentations.items():
            try:
                augmented = transform(image=image, bboxes=bboxes, class_labels=class_labels)
                augmented_image = augmented['image']
                augmented_bboxes = augmented['bboxes']
                augmented_labels = augmented['class_labels']

                # Debugging: Print bounding boxes before and after augmentation
                print(f"{aug_name} - Before: {bboxes}")
                print(f"{aug_name} - After: {augmented_bboxes}")

                # Skip if no bounding boxes remain after transformation
                if not augmented_bboxes:
                    print(f"Skipping {image_name}_{aug_name}: No valid bounding boxes after augmentation.")
                    continue

                # Save augmented image and label
                output_image_path = os.path.join(output_image_dir, f"{image_name.split('.')[0]}_{aug_name}.jpg")
                output_label_path = os.path.join(output_label_dir, f"{image_name.split('.')[0]}_{aug_name}.txt")

                cv2.imwrite(output_image_path, augmented_image)
                write_yolo_label(output_label_path, augmented_bboxes, augmented_labels)
            except Exception as e:
                print(f"Error during {aug_name} augmentation: {e}")
print("Comprehensive data augmentation complete!")

/mnt/nvme_disk2/User_data/hp927k/miniconda3/envs/harsh/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/mnt/nvme_disk2/User_data/hp927k/miniconda3/envs/harsh/lib/python3.10/site-packages/albumentations/core/composition.py:250: UserWarning: Got processor for bboxes, but no transform to process it.
  self._set_keys()


Processing: A A - 3mensio Screen Recording_longitudinal_view_471.png
brightness_contrast - Before: [[0.589073, 0.348885, 0.214476, 0.070432], [0.58839, 0.429074, 0.213483, 0.073615], [0.588989, 0.509865, 0.214532, 0.069371]]
brightness_contrast - After: [[0.5890730023384094, 0.3488849997520447, 0.21447598934173584, 0.07043200731277466], [0.5883899927139282, 0.4290739893913269, 0.21348297595977783, 0.07361501455307007], [0.5889890193939209, 0.5098649859428406, 0.2145320177078247, 0.06937098503112793]]
blur - Before: [[0.589073, 0.348885, 0.214476, 0.070432], [0.58839, 0.429074, 0.213483, 0.073615], [0.588989, 0.509865, 0.214532, 0.069371]]
blur - After: [[0.5890730023384094, 0.3488849997520447, 0.21447598934173584, 0.07043200731277466], [0.5883899927139282, 0.4290739893913269, 0.21348297595977783, 0.07361501455307007], [0.5889890193939209, 0.5098649859428406, 0.2145320177078247, 0.06937098503112793]]
clahe - Before: [[0.589073, 0.348885, 0.214476, 0.070432], [0.58839, 0.429074, 0.213483

libpng error: Read Error
libpng error: Read Error
libpng error: Read Error


brightness_contrast - Before: [[0.54551, 0.358186, 0.176549, 0.055903], [0.532569, 0.427022, 0.173176, 0.06935], [0.532569, 0.500009, 0.166431, 0.049693]]
brightness_contrast - After: [[0.5455099791288376, 0.3581860065460205, 0.17654898762702942, 0.05590301752090454], [0.5325690060853958, 0.4270220100879669, 0.17317602038383484, 0.06935000419616699], [0.5325689762830734, 0.5000090003013611, 0.1664309799671173, 0.04969298839569092]]
blur - Before: [[0.54551, 0.358186, 0.176549, 0.055903], [0.532569, 0.427022, 0.173176, 0.06935], [0.532569, 0.500009, 0.166431, 0.049693]]
blur - After: [[0.5455099791288376, 0.3581860065460205, 0.17654898762702942, 0.05590301752090454], [0.5325690060853958, 0.4270220100879669, 0.17317602038383484, 0.06935000419616699], [0.5325689762830734, 0.5000090003013611, 0.1664309799671173, 0.04969298839569092]]
clahe - Before: [[0.54551, 0.358186, 0.176549, 0.055903], [0.532569, 0.427022, 0.173176, 0.06935], [0.532569, 0.500009, 0.166431, 0.049693]]
clahe - After: [[

libpng error: Read Error
libpng error: Read Error


brightness_contrast - Before: [[0.529214, 0.123714, 0.33983, 0.094203], [0.529763, 0.241721, 0.343087, 0.123188], [0.530303, 0.373188, 0.344167, 0.100399]]
brightness_contrast - After: [[0.5292140245437622, 0.12371399998664856, 0.33983004093170166, 0.09420299530029297], [0.5297629833221436, 0.241721011698246, 0.34308695793151855, 0.12318800389766693], [0.5303030014038086, 0.37318798899650574, 0.34416699409484863, 0.10039901733398438]]
blur - Before: [[0.529214, 0.123714, 0.33983, 0.094203], [0.529763, 0.241721, 0.343087, 0.123188], [0.530303, 0.373188, 0.344167, 0.100399]]
blur - After: [[0.5292140245437622, 0.12371399998664856, 0.33983004093170166, 0.09420299530029297], [0.5297629833221436, 0.241721011698246, 0.34308695793151855, 0.12318800389766693], [0.5303030014038086, 0.37318798899650574, 0.34416699409484863, 0.10039901733398438]]
clahe - Before: [[0.529214, 0.123714, 0.33983, 0.094203], [0.529763, 0.241721, 0.343087, 0.123188], [0.530303, 0.373188, 0.344167, 0.100399]]
clahe - Af

In [ ]:
# import os
# import cv2
# import matplotlib.pyplot as plt

# # Paths to images and labels
# iamge_dir = "/mnt/nvme_disk2/User_data/hp927k/phase2/BBPrediction/images/train_augmented"
# label_dir = "/mnt/nvme_disk2/User_data/hp927k/phase2/BBPrediction/labels/train_augmented"
# output_dir = "/mnt/nvme_disk2/User_data/hp927k/phase2/BBPrediction/yolo_data1/images/visualized"
# os.makedirs(output_dir, exist_ok=True)

# # Get all image filenames
# image_filenames = [f for f in os.listdir(iamge_dir) if f.endswith(('.jpg', '.png', '.jpeg'))]

# # Class names (Modify if needed)
# class_names = {0: "Class 0", 1: "Class 1", 2: "Class 2"}

# # Function to draw bounding boxes
# def draw_boxes(image_path, label_path, output_path):
#     image = cv2.imread(image_path)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     h, w, _ = image.shape
    
#     # Read label file
#     if not os.path.exists(label_path):
#         print(f"No label file for {image_path}")
#         return
    
#     with open(label_path, "r") as f:
#         lines = f.readlines()
    
#     for line in lines:
#         values = line.strip().split()
#         class_id = int(float(values[0]))
#         x_center, y_center, width, height = map(float, values[1:])
        
#         # Convert to pixel coordinates
#         x1 = int((x_center - width / 2) * w)
#         y1 = int((y_center - height / 2) * h)
#         x2 = int((x_center + width / 2) * w)
#         y2 = int((y_center + height / 2) * h)
        
#         # Draw bounding box
#         cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)
#         label = class_names.get(class_id, str(class_id))
#         cv2.putText(image, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
    
#     # Save image
#     output_file = os.path.join(output_path, os.path.basename(image_path))
#     cv2.imwrite(output_file, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
#     print(f"Saved: {output_file}")

# # Visualize first 5 images
# for image_filename in image_filenames[:10]:
#     image_path = os.path.join(iamge_dir, image_filename)
#     label_path = os.path.join(label_dir, os.path.splitext(image_filename)[0] + ".txt")
#     draw_boxes(image_path, label_path, output_dir)


Saved: /mnt/nvme_disk2/User_data/hp927k/phase2/BBPrediction/yolo_data1/images/visualized/A A - 3mensio Screen Recording_longitudinal_view_471.png
Saved: /mnt/nvme_disk2/User_data/hp927k/phase2/BBPrediction/yolo_data1/images/visualized/A A - 3mensio Screen Recording_longitudinal_view_471_brightness_contrast.jpg
Saved: /mnt/nvme_disk2/User_data/hp927k/phase2/BBPrediction/yolo_data1/images/visualized/A A - 3mensio Screen Recording_longitudinal_view_471_blur.jpg
Saved: /mnt/nvme_disk2/User_data/hp927k/phase2/BBPrediction/yolo_data1/images/visualized/A A - 3mensio Screen Recording_longitudinal_view_471_clahe.jpg
Saved: /mnt/nvme_disk2/User_data/hp927k/phase2/BBPrediction/yolo_data1/images/visualized/A A - 3mensio Screen Recording_longitudinal_view_471_scaling.jpg
Saved: /mnt/nvme_disk2/User_data/hp927k/phase2/BBPrediction/yolo_data1/images/visualized/A A - 3mensio Screen Recording_longitudinal_view_474.png
Saved: /mnt/nvme_disk2/User_data/hp927k/phase2/BBPrediction/yolo_data1/images/visuali

In [2]:
import torch

print(f"Available GPUs: {torch.cuda.device_count()}")
print(f"Current Device: {torch.cuda.current_device()}")
print(f"Device Name: {torch.cuda.get_device_name(torch.cuda.current_device())}")
# Set Device
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")  # Debugging: Print device


Available GPUs: 8
Current Device: 0
Device Name: NVIDIA H100 80GB HBM3
Using device: cuda:2


In [6]:
from ultralytics import YOLO
import torch

# Load the YOLOv8 model (Choose 'yolov8n.pt', 'yolov8s.pt', etc.)
model = YOLO("yolov8n.pt").to(device)  # Smallest YOLOv8 model, replace with your choice
# model.to(device="cuda:2")
# Train the model
# Train Model
model.train(
    data="/mnt/nvme_disk2/User_data/hp927k/phase2/BBPrediction/data.yaml",
    epochs=20,
    imgsz=640,
    batch=4,
    device=device,  # Explicitly use GPU 2
    workers=4,
    cache=True,
    name="largerDatasetTraining20epoch"
)

New https://pypi.org/project/ultralytics/8.3.80 available 😃 Update with 'pip install -U ultralytics'
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/mnt/nvme_disk2/User_data/hp927k/phase2/BBPrediction/data.yaml, epochs=20, time=None, patience=100, batch=4, imgsz=640, save=True, save_period=-1, cache=True, device=cuda:2, workers=4, project=None, name=largerDatasetTraining20epoch, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, 

train: Scanning /mnt/nvme_disk2/User_data/hp927k/phase2/BBPrediction/labels/train_augmented.cache... 1900 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1900/1900 [00:00<?, ?it/s]

WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.



train: Caching images (2.0GB RAM): 100%|██████████| 1900/1900 [00:00<00:00, 2230.24it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


...

                   all         99        297      0.688      0.631       0.63      0.216



20 epochs completed in 0.122 hours.
Optimizer stripped from runs/detect/largerDatasetTraining20epoch/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/largerDatasetTraining20epoch/weights/best.pt, 6.2MB

Validating runs/detect/largerDatasetTraining20epoch/weights/best.pt...
Ultralytics 8.3.78 🚀 Python-3.10.16 torch-2.5.1 CUDA:2 (NVIDIA H100 80GB HBM3, 80995MiB)
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:00<00:00, 23.04it/s]


                   all         99        297      0.708      0.814      0.727      0.279
       ascending_aorta         99         99      0.427      0.616      0.444      0.137
                   SOV         99         99       0.88       0.97      0.954      0.493
                  LVOT         99         99      0.817      0.856      0.784      0.205
Speed: 0.1ms preprocess, 1.8ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs/detect/largerDatasetTraining20epoch


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x78aa9a799480>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04

# Drawing Bounding Boxes

In [1]:
import torch
import cv2
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO

# Set device to GPU 1
device = torch.device("cuda:1")

# Load the trained YOLOv8 model (Ensure training is completed)
model = YOLO("/mnt/nvme_disk2/User_data/hp927k/phase2/finalModelPrediction/best.pt").to(device)

# Path to validation image folder
val_images_path = "/mnt/nvme_disk2/User_data/hp927k/phase2/finalModelPrediction/val"

# Path to save output images
output_dir = "/mnt/nvme_disk2/User_data/hp927k/phase2/finalModelPrediction/output_visualization_new100_old_model"
os.makedirs(output_dir, exist_ok=True)  # Create folder if it doesn’t exist

# Get list of all images in validation folder
all_images = [os.path.join(val_images_path, img) for img in os.listdir(val_images_path) if img.endswith((".jpg", ".png", ".jpeg"))]

# Randomly select 10 images
random.seed(42)  # Ensure reproducibility
test_images = random.sample(all_images, min(500, len(all_images)))  # If fewer than 10 images exist, take all

# Class name mapping
class_names = {0: "ascending_aorta", 1: "SOV", 2: "LVOT"}

# Function to filter bounding boxes
def filter_bboxes(results):
    filtered_results = []

    for result in results:
        max_conf_bboxes = {}  # Dictionary to store max-confidence bbox for each class
        valid_bboxes = []

        for box in result.boxes.data:  # YOLOv8 output: tensor [x1, y1, x2, y2, conf, class]
            x1, y1, x2, y2, conf, cls = box.tolist()
            cls = int(cls)

            if conf > 0:  # Consider only boxes with confidence score
                if cls not in max_conf_bboxes or conf > max_conf_bboxes[cls][4]:
                    max_conf_bboxes[cls] = [x1, y1, x2, y2, conf, cls]
            else:
                valid_bboxes.append([x1, y1, x2, y2, cls])  # Boxes without confidence scores

        # Merge max-confidence and valid bounding boxes
        filtered_results.extend(max_conf_bboxes.values())
        filtered_results.extend(valid_bboxes)

    return filtered_results

# Function to draw bounding boxes on an image and save it
def draw_and_save_bboxes(image_path, filtered_bboxes, output_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB

    for bbox in filtered_bboxes:
        x1, y1, x2, y2 = int(bbox[0]), int(bbox[1]), int(bbox[2]), int(bbox[3])
        cls = int(bbox[-1])

        # Draw bounding box
        color = (0, 255, 0)  # Green color
        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)

        # Label with confidence (if available)
        label = class_names.get(cls, "Unknown")
        if len(bbox) == 6:
            label += f" ({bbox[4]:.2f})"

        cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Save the processed image
    output_filename = os.path.basename(image_path).split('.')[0] + "_bbox.png"
    output_filepath = os.path.join(output_dir, output_filename)
    cv2.imwrite(output_filepath, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))  # Convert back to BGR before saving

    print(f"Saved: {output_filepath}")  # Log the saved image

# Run inference and save the images
for image_path in test_images:
    results = model.predict(image_path, conf=0.25, iou=0.5, device=device)
    filtered_bboxes = filter_bboxes(results)
    draw_and_save_bboxes(image_path, filtered_bboxes, output_dir)

WARNING ⚠️ torchvision==0.21 is incompatible with torch==2.5.
Run 'pip install torchvision==0.20' to fix torchvision or 'pip install -U torch torchvision' to update both.
For a full compatibility table see https://github.com/pytorch/vision#installation

image 1/1 /mnt/nvme_disk2/User_data/hp927k/phase2/finalModelPrediction/val/V A F_longitudinal_view_531.png: 640x576 1 ascending_aorta, 1 SOV, 1 LVOT, 67.4ms
Speed: 5.7ms preprocess, 67.4ms inference, 174.0ms postprocess per image at shape (1, 3, 640, 576)
Saved: /mnt/nvme_disk2/User_data/hp927k/phase2/finalModelPrediction/output_visualization_new100_old_model/V A F_longitudinal_view_531_bbox.png

image 1/1 /mnt/nvme_disk2/User_data/hp927k/phase2/finalModelPrediction/val/S G_longitudinal_view_325.png: 640x576 1 ascending_aorta, 1 SOV, 1 LVOT, 3.8ms
Speed: 2.6ms preprocess, 3.8ms inference, 12.0ms postprocess per image at shape (1, 3, 640, 576)
Saved: /mnt/nvme_disk2/User_data/hp927k/phase2/finalModelPrediction/output_visualization_new100

# Implement gradCam